Goal of analysis script is to pull in excel files with manually entered intake and weight data, determine daily intake per body weight (for both water and ethanol) for subsequent analysis


In [39]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import re
import os

%matplotlib inline
import seaborn as sns
import warnings; warnings.simplefilter('ignore')

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/Round 3/mice


In [84]:
path_names = []
files = os.listdir(root.directory)
for file in files: 
        path_names.append(root.directory + "/" + file)

path_names[0]

'C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/Round 3/mice/561.xlsx'

In [85]:
group_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/Round 3/groups.xlsx"
data = pd.read_excel(group_path)
df_mice = pd.DataFrame(data = data)
df_mice.head()

,Animal,Cage_#,Treatment,Number,Group,Notes,Blood loc,2BC,stim,Starting_weight,PostInjury_weight,sac_weight
0,560,177,2,3,5-,sac'd,b,n,n,29.0,29,NaN
1,561,177,2,3,5,belly wounds healing,b,y,y,29.5,26,37.0
2,562,177,2,3,5,NaN,b,y,y,31.6,30,29.9
3,563,178,1,3,4,NaN,b,y,n,27.5,25,37.0
4,564,178,1,3,4,NaN,b,y,n,28.4,27,29.0


Loop through path names to create pandas data table from excel file. Get animal number from path name (animal number is file name), create array of animal number the length of the data_table and add this as a column.

In [86]:
data_start = pd.DataFrame()

for path in path_names:
    data_excel = pd.read_excel(path)
    data_table_int = pd.DataFrame(data = data_excel)
    
    name = path
    name = re.search("\d\d\d", path).group(0)
    name_long = [name] * len(data_table_int)
    
    group = df_mice.loc[df_mice['Animal'] == int(name), 'Group'].item()
    group_long = [group] * len(data_table_int)
    
    data_table_int = data_table_int.assign(Animal = name_long)
    data_table_int = data_table_int.assign(Group = group_long)  
    
    data_start = data_start.append(data_table_int, ignore_index=True)


data_start.head()

,Animal,Day,Day #,EtOH,EtOH 8,EtOH after,EtOH bf,Group,H20 8,H2O after,H2O b4,Notes,Weight
0,561,Friday,1,0,x,79.8,89.1,5,x,81.4,93.8,NaN,33.6
1,561,Monday,2,3,x,59.7,65.0,5,x,78.9,81.4,NaN,33.6
2,561,Tuesday,3,3,x,54.6,59.7,5,x,77.4,78.9,NaN,33.6
3,561,Wednesday,4,3,x,50.2,54.6,5,x,74.1,77.4,NaN,33.6
4,561,Thursday,5,3,49,45.5,50.2,5,73.4,71.8,74.1,NaN,33.6


In [87]:
def C2BC(data_start):
    "Analyzes intermittent two bottle choice data"
    
    data_start['H2O_intake_raw'] = data_start['H2O b4'] - data_start['H2O after']

    data_start['H2O_intake'] = data_start['H2O_intake_raw'] / (data_start['Weight'] /1000)

    data_start['H2O_intake_raw_8'] = data_start['H2O b4'].where(data_start['H20 8'] != 'x') \
    - data_start['H20 8'].where(data_start['H20 8'] != 'x')

    data_start['H2O_intake_8'] = data_start['H2O_intake_raw_8'] / (data_start['Weight'] /1000)



    data_start['EtOH_intake_raw'] = data_start['EtOH bf'] - data_start['EtOH after']

    data_start['EtOH_intake'] = (data_start['EtOH_intake_raw'] * data_start['EtOH']/100) / (data_start['Weight'] /1000)

    data_start['EtOH_intake_raw_8'] = \
    data_start['EtOH bf'].where(data_start['EtOH 8'] != 'x') - data_start['EtOH 8'].where(data_start['EtOH 8'] != 'x')

    data_start['EtOH_intake_8'] = \
    (data_start['EtOH_intake_raw_8'] * data_start['EtOH']/100) / (data_start['Weight'] /1000)


    data_start = data_start.fillna(value = 0)
    
    data_start['EtOH_pref'] = \
    data_start['EtOH_intake_raw'] / (data_start['EtOH_intake_raw'] + data_start['H2O_intake_raw'])

    data_start['EtOH_pref_8'] = \
    data_start['EtOH_intake_raw_8'] / (data_start['EtOH_intake_raw_8'] + data_start['H2O_intake_raw_8'])
    
    
    return data_start

In [88]:
intake_data = data_start.pipe(C2BC)
intake_data.head()

,Animal,Day,Day #,EtOH,EtOH 8,EtOH after,EtOH bf,Group,H20 8,H2O after,H2O b4,Notes,Weight,H2O_intake_raw,H2O_intake,H2O_intake_raw_8,H2O_intake_8,EtOH_intake_raw,EtOH_intake,EtOH_intake_raw_8,EtOH_intake_8,EtOH_pref,EtOH_pref_8
0,561,Friday,1,0,x,79.8,89.1,5,x,81.4,93.8,0.0,33.6,12.4,369.047619,0.0,0.000000,9.3,0.000000,0.0,0.000000,0.428571,NaN
1,561,Monday,2,3,x,59.7,65.0,5,x,78.9,81.4,0.0,33.6,2.5,74.404762,0.0,0.000000,5.3,4.732143,0.0,0.000000,0.679487,NaN
2,561,Tuesday,3,3,x,54.6,59.7,5,x,77.4,78.9,0.0,33.6,1.5,44.642857,0.0,0.000000,5.1,4.553571,0.0,0.000000,0.772727,NaN
3,561,Wednesday,4,3,x,50.2,54.6,5,x,74.1,77.4,0.0,33.6,3.3,98.214286,0.0,0.000000,4.4,3.928571,0.0,0.000000,0.571429,NaN
4,561,Thursday,5,3,49,45.5,50.2,5,73.4,71.8,74.1,0.0,33.6,2.3,68.452381,0.7,20.833333,4.7,4.196429,1.2,1.071429,0.671429,0.631579


In [89]:
intake_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 23 columns):
Animal               396 non-null object
Day                  396 non-null object
Day #                396 non-null int64
EtOH                 396 non-null int64
EtOH 8               396 non-null object
EtOH after           396 non-null float64
EtOH bf              396 non-null float64
Group                396 non-null int64
H20 8                396 non-null object
H2O after            396 non-null float64
H2O b4               396 non-null float64
Notes                396 non-null float64
Weight               396 non-null float64
H2O_intake_raw       396 non-null float64
H2O_intake           396 non-null float64
H2O_intake_raw_8     396 non-null float64
H2O_intake_8         396 non-null float64
EtOH_intake_raw      396 non-null float64
EtOH_intake          396 non-null float64
EtOH_intake_raw_8    396 non-null float64
EtOH_intake_8        396 non-null float64
EtOH_pref            39

In [90]:
data_table_all_calc = intake_data[['Animal', 'Group', 'Day #', 'EtOH', 'H2O_intake', 'H2O_intake_8', 'EtOH_intake', 'EtOH_intake_8', 'EtOH_pref', 'EtOH_pref_8']]
data_table_all_calc.head()

,Animal,Group,Day #,EtOH,H2O_intake,H2O_intake_8,EtOH_intake,EtOH_intake_8,EtOH_pref,EtOH_pref_8
0,561,5,1,0,369.047619,0.000000,0.000000,0.000000,0.428571,NaN
1,561,5,2,3,74.404762,0.000000,4.732143,0.000000,0.679487,NaN
2,561,5,3,3,44.642857,0.000000,4.553571,0.000000,0.772727,NaN
3,561,5,4,3,98.214286,0.000000,3.928571,0.000000,0.571429,NaN
4,561,5,5,3,68.452381,20.833333,4.196429,1.071429,0.671429,0.631579


In [91]:
negatives = data_table_all_calc[data_table_all_calc['EtOH'] < 0]
negatives.head()

,Animal,Group,Day #,EtOH,H2O_intake,H2O_intake_8,EtOH_intake,EtOH_intake_8,EtOH_pref,EtOH_pref_8


In [92]:
data_table_calc = data_table_all_calc
data_table_calc.groupby(['Group', 'EtOH']).describe()

Day #                                               EtOH_intake                                                                           EtOH_intake_8                                                          EtOH_pref                                                                       EtOH_pref_8                                                                       H2O_intake                                                                                           H2O_intake_8                                                          
           count  mean       std  min   25%   50%    75%   max       count       mean       std       min       25%        50%        75%        max         count      mean        std  min  25%  50%       75%        max     count      mean       std       min       25%       50%       75%       max       count      mean       std       min       25%       50%       75%       max      count         mean           std         min         25%         50%         75%            max        count      mean        std  min  25%  50%        75%        max
Group EtOH                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2     0      6.0   1.0  0.000000  1.0  1.00   1.0   1.00   1.0         6.0   0.000000  0.000000  0.000000  0.000000   0.000000   0.000000   0.000000           6.0  0.000000   0.000000  0.0  0.0  0.0  0.000000   0.000000       6.0  0.524902  0.232135  0.251938  0.328196  0.553507  0.660967  0.840580         0.0       NaN       NaN       NaN       NaN       NaN       NaN       NaN        6.0   272.011058    200.935213   78.014184  158.863414  220.672129  301.889866     645.484950          6.0  0.000000   0.000000  0.0  0.0  0.0   0.000000   0.000000
      3     24.0   3.5  1.142080  2.0  2.75   3.5   4.25   5.0        24.0   3.252999  0.844867  1.595745  2.502635   3.419299   3.952128   4.782609          24.0  0.127808   0.258035  0.0  0.0  0.0  0.026596   0.868167      24.0  0.616944  0.139470  0.326087  0.490474  0.635467  0.744310  0.804348         6.0  0.488456  0.234249  0.142857  0.375000  0.500000  0.602273  0.818182       24.0    66.740590     24.128830   31.914894   43.246994   69.512328   81.506382     110.367893         24.0  4.126248   7.820496  0.0  0.0  0.0   1.607717  23.188406
      6     18.0   7.0  0.840168  6.0  6.00   7.0   8.00   8.0        18.0   7.898830  1.713105  4.680851  6.781480   8.094000   8.608028  12.688172          18.0  0.607465   0.942815  0.0  0.0  0.0  1.198604   2.580645      18.0  0.726288  0.088881  0.458333  0.697861  0.734492  0.781340  0.852459         6.0  0.703579  0.062249  0.625000  0.671875  0.689904  0.735577  0.800000       18.0    49.914791     19.369166   26.086957   36.016035   47.145720   52.760042      93.189964         18.0  4.045662   6.052145  0.0  0.0  0.0  10.390299  16.722408
      12    24.0  10.5  1.142080  9.0  9.75  10.5  11.25  12.0        24.0  10.821867  4.048292  4.342105  7.599889  10.687096  13.902439  21.261261          24.0  0.687619   1.301872  0.0  0.0  0.0  0.296053   4.181185      24.0  0.538460  0.180194  0.207547  0.379828  0.578784  0.657051  0.819444         6.0  0.512611  0.197809  0.200000  0.401070  0.585227  0.625000  0.727273       24.0    79.440774     37.276041   30.030030   53.178729   71.519280  102.542373     158.450704         24.0  5.861493  11.833867  0.0  0.0  0.0   2.252252  39.473684
3     0      6.0   1.0  0.000000  1.0  1.00   1.0   1.00   1.0         6.0   0.000000  0.000000  0.000000  0.000000   0.000000   0.00

In [93]:
data_table_calc.columns.values

array(['Animal', 'Group', 'Day #', 'EtOH', 'H2O_intake', 'H2O_intake_8',
       'EtOH_intake', 'EtOH_intake_8', 'EtOH_pref', 'EtOH_pref_8'],
      dtype=object)

In [94]:
EtOH_days = pd.DataFrame(data = data_table_calc[['Animal', 'Group', 'Day #', 'EtOH', 'EtOH_intake', 'EtOH_intake_8', 'EtOH_pref', 'EtOH_pref_8']])
EtOH_days.head()

,Animal,Group,Day #,EtOH,EtOH_intake,EtOH_intake_8,EtOH_pref,EtOH_pref_8
0,561,5,1,0,0.000000,0.000000,0.428571,NaN
1,561,5,2,3,4.732143,0.000000,0.679487,NaN
2,561,5,3,3,4.553571,0.000000,0.772727,NaN
3,561,5,4,3,3.928571,0.000000,0.571429,NaN
4,561,5,5,3,4.196429,1.071429,0.671429,0.631579


In [95]:
EtOH_days = EtOH_days.set_index(['Group', 'Animal', 'Day #']).unstack(level=-1)
#EtOH_days.groupby(['Group']).mean()
EtOH_days.dropna(inplace=True, axis = 1)
EtOH_days.head()

EtOH                                      EtOH_intake                                                                                                                   EtOH_intake_8                                                                       EtOH_pref                                                                                                               EtOH_pref_8                    
Day #          1  2  3  4  5  6  7  8   9   10  11  12          1         2         3         4         5         6         7         8          9          10         11         12            1    2    3    4         5    6    7         8    9    10   11        12        1         2         3         4         5         6         7         8         9         10        11        12          5         8         12
Group Animal                                                                                                                                                                                                                                                                                                                                                                                                                    
2     572       0  3  3  3  3  6  6  6  12  12  12  12         0.0  3.723404  3.404255  3.936170  1.595745  5.957447  4.680851  8.085106  10.563380  12.676056   8.873239   6.760563           0.0  0.0  0.0  0.0  0.106383  0.0  0.0  1.276596  0.0  0.0  0.0  2.535211  0.840580  0.744681  0.744186  0.804348  0.326087  0.736842  0.458333  0.808511  0.490196  0.576923  0.318182  0.340426    0.142857  0.666667  0.545455
      573       0  3  3  3  3  6  6  6  12  12  12  12         0.0  4.147910  2.122186  3.376206  4.147910  6.752412  8.102894  8.681672  21.261261   5.045045  14.414414  10.810811           0.0  0.0  0.0  0.0  0.868167  0.0  0.0  0.964630  0.0  0.0  0.0  2.882883  0.625000  0.796296  0.458333  0.673077  0.796296  0.714286  0.807692  0.818182  0.819444  0.583333  0.800000  0.750000    0.818182  0.625000  0.727273
      574       0  3  3  3  3  6  6  6  12  12  12  12         0.0  4.000000  2.260870  4.782609  2.434783  9.043478  8.347826  8.695652  12.786885  13.114754   7.213115   9.836066           0.0  0.0  0.0  0.0  0.347826  0.0  0.0  2.086957  0.0  0.0  0.0  3.278689  0.672956  0.766667  0.490566  0.785714  0.528302  0.852459  0.786885  0.746269  0.722222  0.666667  0.379310  0.555556    0.333333  0.750000  0.625000
      578       0  3  3  3  3  6  6  6  12  12  12  12         0.0  4.214047  3.913043  2.709030  3.612040  7.826087  8.227425  9.632107  12.631579   4.736842   7.105263   4.342105           0.0  0.0  0.0  0.0  0.702341  0.0  0.0  2.207358  0.0  0.0  0.0  1.184211  0.251938  0.626866  0.684211  0.450000  0.679245  0.709091  0.732143  0.716418  0.653061  0.214286  0.327273  0.207547    0.636364  0.687500  0.200000
      579       0  3  3  3  3  6  6  6  12  12  12  12         0.0  2.121212  3.434343  3.030303  2.525253  6.464646  7.070707  6.868687  14.237288  10.169492   8.135593   7.728814           0.0  0.0  0.0  0.0  0.505051  0.0  0.0  1.818182  0.0  0.0  0.0  2.440678  0.276923  0.466667  0.693878  0.600000  0.510204  0.680851  0.744681  0.693878  0.648148  0.500000  0.400000  0.380000    0.500000  0.692308  0.352941

In [96]:
EtOH_days.to_csv('EtOH_days.csv')

In [97]:
sns.barplot(x = 'EtOH', y = 'EtOH_intake_8', hue = 'Group', data = EtOH_days, alpha=0.9)

ValueError: Could not interpret input 'Group'